# ETL Flood and Crime Data

In [1]:
#The following code takes the flood data from csv files extracted from APIs, transforms the data and loads it in SQL
# It also extracts crime data, transforms it and load it in SQl

In [2]:
# Import Dependencies
import pandas as pd
import scipy.stats as st
import gmaps
import numpy as np
import os

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect,join
from secret import username, password
from sqlalchemy.types import Integer, Text, String, Float, DateTime

# Flood Data

### Extract Flood Data Files

In [3]:
entries=os.listdir("Flood_Data/")
entries

['flood_data_77002.csv',
 'flood_data_77006.csv',
 'flood_data_77019.csv',
 'flood_data_77025.csv',
 'flood_data_77027.csv',
 'flood_data_77030.csv',
 'flood_data_77054.csv',
 'flood_data_77098.csv',
 'flood_data_77284.csv',
 'flood_data_77535.csv',
 'flood_data_batch0.csv',
 'flood_data_batch1.csv',
 'flood_data_batch2.csv',
 'flood_data_batch3.csv',
 'flood_data_batch4.csv',
 'flood_data_batch5.csv',
 'flood_data_batch6.csv',
 'flood_data_batch7.csv',
 'flood_data_batch8.csv']

## Transformation

In [4]:
# read all files with flood data
flood_data_df=pd.DataFrame()

for entry in entries:
    file = (f'Flood_Data/{entry}')
    new_data = pd.read_csv(file)
    frames=[flood_data_df, new_data]
    flood_data_df=pd.concat(frames)    

#Remove all duplicate addresses
flood_data_df=flood_data_df.drop_duplicates(subset="Address")
flood_data_df=flood_data_df.reset_index(drop=True)
del flood_data_df['Unnamed: 0']
flood_data_df=flood_data_df.rename(columns={"Flood Description": "Flood_Description", "Flood Zone":"Flood_Zone"})
flood_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23230 entries, 0 to 23229
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Address            23230 non-null  object 
 1   Latitude           23230 non-null  float64
 2   Longitude          23230 non-null  float64
 3   Flood_Description  19160 non-null  object 
 4   Flood_Zone         23230 non-null  object 
dtypes: float64(2), object(3)
memory usage: 907.5+ KB


In [5]:
#Add flood ranking
#3- High Risk
#2 - Medium Risk
#1- Low Risk

flood_data_df['Flood_Risk']=np.where(flood_data_df['Flood_Description']=='AREA OF MINIMAL FLOOD HAZARD',1," ")
flood_data_df.loc[(flood_data_df['Flood_Description']=='0.2 PCT ANNUAL CHANCE FLOOD HAZARD'),'Flood_Risk']=2
flood_data_df.loc[(flood_data_df['Flood_Zone']=='AE'),'Flood_Risk']=3
flood_data_df.reset_index(drop=True)
flood_data_df.sample(5)

,Address,Latitude,Longitude,Flood_Description,Flood_Zone,Flood_Risk
2406,"2239 MIMOSA DR HOUSTON, Texas 77019",29.74737,-95.41465,AREA OF MINIMAL FLOOD HAZARD,X,1
5433,"2308 MCDUFFIE ST HOUSTON, Texas 77019",29.74526,-95.40801,AREA OF MINIMAL FLOOD HAZARD,X,1
10527,"4023 LEESHIRE DR HOUSTON, Texas 77025",29.68171,-95.44197,NaN,AE,3
6517,"3815 MERRICK ST HOUSTON, Texas 77025",29.69658,-95.43811,NaN,AE,3
7398,"3122 BLUE BONNET BLVD HOUSTON, Texas 77025",29.70080,-95.42708,NaN,AE,3


In [6]:
#Generate a colum that has SITE_ADDR_1 to Merge with HCAD Data
address_split=flood_data_df['Address'].str.rsplit(' ',3).tolist()
address_split=pd.DataFrame(address_split)
flood_data_df['SITE_ADDR_1']= address_split[0]
flood_data_df.sample(5)

,Address,Latitude,Longitude,Flood_Description,Flood_Zone,Flood_Risk,SITE_ADDR_1
972,"1501 VICTOR ST HOUSTON, Texas 77019",29.75382,-95.38231,AREA OF MINIMAL FLOOD HAZARD,X,1,1501 VICTOR ST
13372,"57 BRIAR HOLLOW LN 2 HOUSTON, Texas 77027",29.75609,-95.45129,FLOODWAY,AE,3,57 BRIAR HOLLOW LN 2
10223,"4134 FALKIRK LN HOUSTON, Texas 77025",29.69294,-95.44580,NaN,AE,3,4134 FALKIRK LN
21292,"1916 BAILEY ST, Houston, Texas 77006",29.75109,-95.38228,AREA OF MINIMAL FLOOD HAZARD,X,1,"1916 BAILEY ST,"
19727,"1316 W ALABAMA ST, Houston, Texas 77006",29.73884,-95.39540,AREA OF MINIMAL FLOOD HAZARD,X,1,"1316 W ALABAMA ST,"


In [7]:
#Generate ZIP Code to Merge with Crime Data
address_split=flood_data_df['Address'].str.rsplit(' ',3).tolist()
address_split=pd.DataFrame(address_split)
flood_data_df['Zip_Code']= address_split[3]
flood_data_df.sample(5)

,Address,Latitude,Longitude,Flood_Description,Flood_Zone,Flood_Risk,SITE_ADDR_1,Zip_Code
5361,"2105 DRISCOLL ST HOUSTON, Texas 77019",29.74666,-95.40609,AREA OF MINIMAL FLOOD HAZARD,X,1,2105 DRISCOLL ST,77019
6707,"3606 UNDERWOOD ST HOUSTON, Texas 77025",29.69988,-95.43157,NaN,AE,3,3606 UNDERWOOD ST,77025
5562,"1724 WOODHEAD ST HOUSTON, Texas 77019",29.75006,-95.40489,AREA OF MINIMAL FLOOD HAZARD,X,1,1724 WOODHEAD ST,77019
7018,"4115 TURNBERRY CIR HOUSTON, Texas 77025",29.70035,-95.44390,NaN,AE,3,4115 TURNBERRY CIR,77025
7646,"3518 MERRICK ST HOUSTON, Texas 77025",29.69753,-95.43037,NaN,AE,3,3518 MERRICK ST,77025


In [8]:
flood_data_df.to_csv('flood_data.csv')

## Load Flood Data in SQL PostGres

In [9]:
#Connect to local database
rds_connection_string = f'{username}:{password}@localhost:5432/flood_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
#Creating the table in SQL
table_name='flood_data'

flood_data_df.to_sql(
    table_name,
    engine,
    if_exists='replace',
    index=False,
    chunksize=500,
    dtype={
        "Address": String(100),
        "Latitude": Float,
        "Longitude": Float,
        "Flood_Description":  String(50),
        "Flood_Zone": String(50),
        "Flood_Risk": Integer,
        "SITE_ADDR_1": String(100),
        "Zip_Code": Integer
    })

#Set Primary Key
with engine.connect() as con:
    con.execute('ALTER TABLE "flood_data" ADD PRIMARY KEY ("Address");')

In [11]:
# Confirm that data has been added
pd.read_sql_query('select * from flood_data', con=engine).head()

,Address,Latitude,Longitude,Flood_Description,Flood_Zone,Flood_Risk,SITE_ADDR_1,Zip_Code
0,"2202 CAROLINE ST Houston, Texas 77002",29.74614,-95.36987,AREA OF MINIMAL FLOOD HAZARD,X,1,2202 CAROLINE ST,77002
1,"2204 CAROLINE ST Houston, Texas 77002",29.74619,-95.36996,AREA OF MINIMAL FLOOD HAZARD,X,1,2204 CAROLINE ST,77002
2,"2206 CAROLINE ST Houston, Texas 77002",29.74624,-95.37004,AREA OF MINIMAL FLOOD HAZARD,X,1,2206 CAROLINE ST,77002
3,"2251 AUSTIN ST Houston, Texas 77002",29.74530,-95.36882,AREA OF MINIMAL FLOOD HAZARD,X,1,2251 AUSTIN ST,77002
4,"2255 AUSTIN ST Houston, Texas 77002",29.74525,-95.36874,AREA OF MINIMAL FLOOD HAZARD,X,1,2255 AUSTIN ST,77002


# Crime Data

## Extract Crime Data Files

In [12]:
# Read Crime Data file
crime_data_df = pd.read_csv("Crime_Data/2019_Houston_Crimes.csv")
crime_data_df.head()

,Incident,Occurrence\nDate,Occurrence\nHour,NIBRS\nClass,NIBRSDescription,Offense Count,Beat,Premise,Block Range,StreetName,Street\nType,Suffix,City,ZIP Code
0,5619,1/1/2019,0,290,"Destruction, damage, vandalism",1,9C30,"Residence, Home (Includes Apartment)",9622,SAN CARLOS,NaN,NaN,HOUSTON,77013
1,17319,1/1/2019,0,35A,"Drug, narcotic violations",1,7C10,"Highway, Road, Street, Alley",NaN,EAST,FWY,NaN,HOUSTON,77020
2,18119,1/1/2019,0,290,"Destruction, damage, vandalism",1,1.60E+41,"Residence, Home (Includes Apartment)",16718,LONE QUAIL,CT,NaN,HOUSTON,77489
3,19019,1/1/2019,0,520,Weapon law violations,1,NaN,"Residence, Home (Includes Apartment)",1909,MELBOURNE,NaN,NaN,HOUSTON,77026-0000
4,20519,1/1/2019,0,13A,Aggravated Assault,1,1.50E+31,"Residence, Home (Includes Apartment)",4034,OSBY,DR,NaN,HOUSTON,77025


## Transformation

In [13]:
#Clean Zip-Code Data
#Get 5 digit Zip Code
crime_data_df[['ZIP Code']]=crime_data_df[['ZIP Code']].astype(str)
zip_split=crime_data_df['ZIP Code'].str.rsplit('-',1).tolist()
zip_split=pd.DataFrame(zip_split)
crime_data_df['Zip_Code']= zip_split[0]

#Delete unecessary columns
del crime_data_df ['ZIP Code']
del crime_data_df ['Beat']
del crime_data_df ['Suffix']

#Clean ZIP Code Columns
crime_data_df=crime_data_df.loc[crime_data_df['Zip_Code']!='nan',:]
crime_data_df['Zip_Code']=crime_data_df['Zip_Code'].astype(int)
crime_data_df = crime_data_df.reset_index(drop=True)

#Rename Columns
crime_data_df=crime_data_df.rename(columns={"Occurrence\nDate": "Date",
                                           "Occurrence\nHour": "Hour",
                                            "NIBRS\nClass":"NIBRS_Class",
                                            "NIBRSDescription":"NIBRS_Description",
                                            "Offense Count":"Offense_Count",
                                            "Block Range":"Block_Range",
                                            "StreetName":"Street_Name",
                                            "Street\nType":"Street_Type"
                                           })

#Create CSV for QC
crime_data_df.to_csv('crime_data.csv')
crime_data_df.head()

,Incident,Date,Hour,NIBRS_Class,NIBRS_Description,Offense_Count,Premise,Block_Range,Street_Name,Street_Type,City,Zip_Code
0,5619,1/1/2019,0,290,"Destruction, damage, vandalism",1,"Residence, Home (Includes Apartment)",9622,SAN CARLOS,NaN,HOUSTON,77013
1,17319,1/1/2019,0,35A,"Drug, narcotic violations",1,"Highway, Road, Street, Alley",NaN,EAST,FWY,HOUSTON,77020
2,18119,1/1/2019,0,290,"Destruction, damage, vandalism",1,"Residence, Home (Includes Apartment)",16718,LONE QUAIL,CT,HOUSTON,77489
3,19019,1/1/2019,0,520,Weapon law violations,1,"Residence, Home (Includes Apartment)",1909,MELBOURNE,NaN,HOUSTON,77026
4,20519,1/1/2019,0,13A,Aggravated Assault,1,"Residence, Home (Includes Apartment)",4034,OSBY,DR,HOUSTON,77025


In [14]:
crime_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216323 entries, 0 to 216322
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Incident           216323 non-null  int64 
 1   Date               216323 non-null  object
 2   Hour               216323 non-null  int64 
 3   NIBRS_Class        216323 non-null  object
 4   NIBRS_Description  216323 non-null  object
 5   Offense_Count      216323 non-null  int64 
 6   Premise            216323 non-null  object
 7   Block_Range        215618 non-null  object
 8   Street_Name        216323 non-null  object
 9   Street_Type        199235 non-null  object
 10  City               216323 non-null  object
 11  Zip_Code           216323 non-null  int32 
dtypes: int32(1), int64(3), object(8)
memory usage: 19.0+ MB


## Load Crime Data in SQL PostGres

In [15]:
#Connect to local database
rds_connection_string = f'{username}:{password}@localhost:5432/flood_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [16]:
#Creating the table in SQL
table_name='crime_data'

crime_data_df.to_sql(
    table_name,
    engine,
    if_exists='replace',
    index=True,
    chunksize=500,
    dtype={
        "Incident": Integer,
        "Date": DateTime,
        "Hour": Integer,
        "NIBRS_Class":  String(100),
        "NIBRS_Description": String(100),
        "Offense_Count": Integer,
        "Premise": String(100),
        "Block_Range": String(20),
        "Street_Name": String(50),
        'Street_Type':String (10),
        "City":String (50),
        "Zip_Code":Integer
    })

#Set Primary Key
with engine.connect() as con:
    con.execute('ALTER TABLE "crime_data" ADD PRIMARY KEY ("index");')

In [17]:
# Confirm that data has been added
pd.read_sql_query('select * from crime_data', con=engine).head()

,index,Incident,Date,Hour,NIBRS_Class,NIBRS_Description,Offense_Count,Premise,Block_Range,Street_Name,Street_Type,City,Zip_Code
0,0,5619,2019-01-01,0,290,"Destruction, damage, vandalism",1,"Residence, Home (Includes Apartment)",9622,SAN CARLOS,None,HOUSTON,77013
1,1,17319,2019-01-01,0,35A,"Drug, narcotic violations",1,"Highway, Road, Street, Alley",None,EAST,FWY,HOUSTON,77020
2,2,18119,2019-01-01,0,290,"Destruction, damage, vandalism",1,"Residence, Home (Includes Apartment)",16718,LONE QUAIL,CT,HOUSTON,77489
3,3,19019,2019-01-01,0,520,Weapon law violations,1,"Residence, Home (Includes Apartment)",1909,MELBOURNE,None,HOUSTON,77026
4,4,20519,2019-01-01,0,13A,Aggravated Assault,1,"Residence, Home (Includes Apartment)",4034,OSBY,DR,HOUSTON,77025
